In [1]:
from sklearn import cross_validation,model_selection
from sklearn import svm  
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn import cross_validation,model_selection
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from sklearn.feature_extraction.text import TfidfTransformer
from string import count
ps = PorterStemmer()

/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/home/umang/.local/lib/python2.7/site-packages

/home/umang/.local/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/home/umang/.local/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/home/umang/.local/lib/python2.7/site-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (lib, index as libindex, tslib as libts,
/home/umang/.local/lib/python2.7/site-packages/pandas/tseries/offsets.py:21: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  import pandas._libs.tslibs.offsets as liboffsets
/home/umang/.local/lib/python2.7/site-packages/pandas/core/ops.py:16: RuntimeWarning: numpy.d

In [2]:
word2vec_features = pd.read_csv("/home/umang/Desktop/AM_Email_Tagging/Intermediate Excel Files/ummang_word2vec.csv")

In [3]:
word2vec_features.head()

,Unnamed: 0,Amcatid,Sentences,Tag_name,Flow,Rater_id,All_tags,cleaned_text,0,1,...,291,292,293,294,295,296,297,298,299,flag
0,0,10016611374320,"I am Rajeev Pathak, CSE,IT Systems",oi_tag,3,1,"['oi', '/oi', 'ccd', '/ccd']",i am rajeev pathak cse it systems,0.003418,-0.187500,...,0.156128,0.002563,0.336914,0.118469,-0.143799,0.098267,-0.203857,-0.007050,0.055481,0
1,1,10016611374320,. I can send you the detailed time and cost es...,ccd_tag,3,1,"['oi', '/oi', 'ccd', '/ccd']",i can send you the detailed time and cost es...,-0.031738,-0.119141,...,0.225403,-0.262939,-0.197815,0.301208,0.294434,0.207520,-0.430069,0.114990,-0.767578,0
2,2,10016614250601,Super Bazaar is looking to upgrade its old in...,op_tag,1,1,"['op', '/op', 'cdf', '/cdf']",super bazaar is looking to upgrade its old in...,0.493530,0.235840,...,0.743286,0.121674,0.478882,0.555786,-0.464111,-0.050537,-0.084595,0.132950,-0.100220,0
3,3,10016614250601,we will tell you our simpler way to deal.\n\n\n,cdf_tag,1,1,"['op', '/op', 'cdf', '/cdf']",we will tell you our simpler way to deal,0.247253,-0.041504,...,0.080811,-0.165283,-0.085449,-0.573730,0.235352,0.309509,-0.439819,-0.047852,0.007416,0
4,4,10016616206567,This is to inform you sir that we offer the mo...,op_tag,2,1,"['op', '/op', 'ccd', '/ccd']",this is to inform you sir that we offer the mo...,-0.264114,-0.421753,...,0.983261,-0.791138,0.642151,0.433533,-0.424728,0.186951,-0.890274,0.104950,-0.127930,0


In [4]:
all_sentences_stemmed = []
all_tags = []
opening_tags = ['op_tag','oi_tag','opc_tag','ow_tag','oa_tag','ocg_tag','odf_tag']
closing_tags = ['cac_tag','cap_tag','ccd_tag','cdf_tag','cff_tag','cth_tag','cw_tag']
i = 0
for row in range(0,word2vec_features.index.size):
    if word2vec_features['Tag_name'][i] in opening_tags or word2vec_features['Tag_name'][i] in closing_tags:
        all_tags.append(word2vec_features['Tag_name'][i])
        splitted_arr = word2vec_features['cleaned_text'][i].split(" ")
        sentence= []
        for word in splitted_arr:
            sentence.append(ps.stem(word))
        sentence = " ".join(sentence)
        word2vec_features = word2vec_features.set_value(row,'Cleaned Stemmed Text', sentence)
    i += 1


/home/umang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [6]:
num_feats = [50,100,150,200,250,300]
models = ['log_reg' , 'svm']
model_list = []
feat_list = []
accuracy_mean_list = []
accuracy_stdev_list = []
report_lr_list = []
for model in models:
    for feat_num in num_feats:
        model_list.append(model)
        feat_list.append(feat_num)
        accuracy_lr = []
        report_lr = []
        feat_list = []
        print model, feat_num
        for i in tqdm(range(0,20)):
            X_train, X_test, y_train,y_test = cross_validation.train_test_split(word2vec_features,word2vec_features['Tag_name'], test_size = 0.30)
            count_vectorizer = CountVectorizer()
            X_train_word2vec = X_train.iloc[:,8:308]
            X_test_word2vec = X_test.iloc[:,8:308]
            X_train_tfidf = count_vectorizer.fit_transform(X_train['Cleaned Stemmed Text'])
            X_test_tfidf = count_vectorizer.transform(X_test['Cleaned Stemmed Text'])
            tfidf_columns = count_vectorizer.vocabulary_.keys()
            X_train_word2vec = X_train_word2vec.reset_index(drop='true')
            X_test_word2vec = X_test_word2vec.reset_index(drop='true')
            tfidf = TfidfTransformer(norm="l2")
            tfidf.fit(X_train_tfidf)
            X_train_tfidf = tfidf.transform(X_train_tfidf).todense()
            X_test_tfidf = tfidf.transform(X_test_tfidf).todense()
            
            X_train_tfidf = pd.DataFrame(X_train_tfidf , columns=tfidf_columns)
            X_test_tfidf = pd.DataFrame(X_test_tfidf , columns=tfidf_columns)

        
            X_train_new = pd.concat([X_train_word2vec,X_train_tfidf],axis=1)
            X_test_new = pd.concat([X_test_word2vec,X_test_tfidf],axis=1)
             
            selectkBest=SelectKBest(f_classif, k=feat_num)
            X_new_train=selectkBest.fit_transform(X_train_new, y_train)
            features_selected = X_train_new.columns[selectkBest.get_support()]
            X_new_test=selectkBest.transform(X_test_new)
            
            if model == 'log_reg':
                model_used = LogisticRegression()
            if model == 'svm':
                model_used = svm.SVC()
            model_used.fit(X_new_train, y_train)
            predictions = model_used.predict(X_new_test)
            accuracy_lr.append(accuracy_score(y_test, predictions))
            y_pred = predictions
            y_true = y_test
            report_lr.append(precision_recall_fscore_support(y_true, y_pred, average= None , labels=opening_tags))
        report_lr = np.mean(report_lr,axis=0)
        precision_ = list(report_lr[0])
        feat_list.append(features_selected)
        precision_.append('precision')
        recall_ = list(report_lr[1])
        recall_.append('recall')
        f1_score_ = list(report_lr[2])
        f1_score_.append('f1_score')
        model_type = [model, feat_num]
        precision_.extend(model_type)
        recall_.extend(model_type)
        f1_score_.extend(model_type)
        report_lr_list.append(precision_)
        report_lr_list.append(recall_)
        report_lr_list.append(f1_score_)
        accuracy_mean_list.append(np.mean(accuracy_lr))
        accuracy_stdev_list.append(np.std(accuracy_lr))

  0%|          | 0/20 [00:00<?, ?it/s]

log_reg 50


  5%|▌         | 1/20 [00:03<00:57,  3.01s/it]/home/umang/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
  0%|          | 0/20 [00:00<?, ?it/s]

log_reg 100


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

svm 250


  0%|          | 0/20 [00:00<?, ?it/s]

svm 300


100%|██████████| 20/20 [11:56<00:00, 35.84s/it]


In [8]:
X_train_new = pd.DataFrame()
X_test_new = pd.DataFrame()
X_train_new = pd.concat([X_train_word2vec,X_train_tfidf],axis=1)
X_test_new = pd.concat([X_test_word2vec,X_test_tfidf],axis=1)
all_features = pd.DataFrame()
all_features = pd.concat([X_test_new,X_test_new],axis=1)
all_features 
all_features.to_csv("/home/umang/Desktop/all_features.csv")

In [9]:
classification_df = pd.DataFrame(report_lr_list,columns=opening_tags+['metric','model','features'])
print classification_df

      op_tag    oi_tag   opc_tag    ow_tag    oa_tag   ocg_tag   odf_tag  \
0   0.567347  0.803834  0.622290  0.108333  0.662464  0.630734  0.365576   
1   0.466320  0.877808  0.702564  0.020892  0.745804  0.684913  0.284433   
2   0.510710  0.838949  0.659475  0.034835  0.701163  0.656295  0.318989   
3   0.612975  0.829397  0.678049  0.416111  0.723119  0.628244  0.416956   
4   0.519417  0.884797  0.728679  0.165850  0.804760  0.706105  0.358065   
5   0.561415  0.855962  0.702185  0.229936  0.761344  0.664056  0.384786   
6   0.633118  0.834489  0.689237  0.633478  0.736723  0.645719  0.438164   
7   0.541250  0.873696  0.744509  0.339734  0.813557  0.719291  0.382460   
8   0.582834  0.853321  0.715032  0.439351  0.772351  0.679623  0.407611   
9   0.624546  0.836469  0.700527  0.726806  0.739155  0.635358  0.444914   
10  0.555155  0.878406  0.742371  0.509693  0.819736  0.726430  0.395181   
11  0.587100  0.856768  0.720539  0.578018  0.776657  0.677211  0.418159   
12  0.638790

In [13]:
accuracy_df = pd.DataFrame()
accuracy_df['model'] = model_list
accuracy_df['accuracy_mean'] = accuracy_mean_list
accuracy_df['accuracy_stdev'] = accuracy_stdev_list
features = pd.DataFrame()


In [15]:
##df.to_csv('/home/umang/Desktop/AM_Email_Tagging_Model_Building_Results/Gap_of_10:200_to_250/Closing_Tags/accuracy.csv')
accuracy_df

,model,accuracy_mean,accuracy_stdev
0,log_reg,0.581063,0.007046
1,log_reg,0.625631,0.007727
2,log_reg,0.640061,0.006373
3,log_reg,0.647360,0.006208
4,log_reg,0.656324,0.008266
5,log_reg,0.656862,0.008696
6,svm,0.588059,0.011096
7,svm,0.621611,0.006886
8,svm,0.624723,0.007241
9,svm,0.629129,0.005858
